In [ ]:
import pandas as pd
import numpy as np
import psycopg2 as ps

In [7]:
# Creamos la conexión a la base de datos
# Vamos a crear una conexión a la base de datos.
conn = ps.connect(
    dbname = "BBDD_Hoteles", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [8]:
# Creamos un cursor el cual nos va a permitir ejecutar querys.
cur = conn.cursor()

In [9]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)

In [10]:
query = """ 
        SELECT * FROM hoteles;
"""

In [12]:
cur.execute(query)
cur.fetchmany(3)

[]

In [13]:
# una vez hemos terminado de trabajar es necesario cerrar la conexión y el cursor.
cur.close()
conn.close()